In [ ]:
import pickle
import pandas as pd
import numpy as np
import torch
train_data = pickle.load(open('embeddings', 'rb'))

In [ ]:
import gc
gc.collect()

In [ ]:
emoji_stats = [
    ('👍', 14189),
    ('\U0001fae1', 76),
    ('🙏', 157),
    ('🔥', 5659),
    ('🥰', 1186),
    ('👌', 69),
    ('🤓', 58),
    ('🤝', 3),
    ('👎', 2049),
    ('🤬', 182),
    ('😁', 2405),
    ('🤡', 1453),
    ('🏆', 290),
    ('👏', 320),
    ('💯', 160),
    ('❤', 4263),
    ('🌚', 254),
    ('🤨', 293),
    ('🥴', 274),
    ('🤩', 238),
    ('😍', 64),
    ('🤣', 348),
    ('😢', 2800),
    ('💩', 1169),
    ('🤯', 598),
    ('❤\u200d🔥', 596),
    ('🐳', 457),
    ('🤮', 1203),
    ('🤗', 2),
    ('😇', 20),
    ('🤔', 964),
    ('🖕', 114),
    ('🥱', 93),
    ('😈', 213),
    ('🕊', 118),
    ('🍌', 476),
    ('🌭', 234),
    ('💋', 219),
    ('⚡', 83),
    ('🍓', 113),
    ('🍾', 287),
    ('💔', 38),
    ('😱', 442),
    ('🎉', 731),
    ('😐', 76),
    ('✍', 34),
    ('😭', 116),
    ('🆒', 31),
    ('🗿', 7),
    ('👀', 48),
    ('💅', 6),
    ('🎄', 66),
    ('☃', 3),
    ('👨\u200d💻', 2),
    ('👻', 7),
    ('🙊', 1),
    ('🤪', 4),
    ('😨', 2),
    ('💊', 2),
    ('😴', 2),
]
class_weights = {}
for emoji_id in range(60):
    class_weights[emoji_id] = emoji_stats[emoji_id][1]

In [ ]:
#!pip install xgboost
import lightgbm
import xgboost
n_estimators = 25
learning_rate = 0.03
model = lightgbm.LGBMClassifier(n_estimators=n_estimators, learning_rate=learning_rate, class_weight=class_weights)

In [ ]:
target_data = pd.read_csv('data_all.csv')

In [ ]:
target_data = target_data.drop(['text', 'total', 'Unnamed: 52'], axis=1)
target_data_np = np.array(target_data)
target = np.argmax(target_data_np, axis=1)
target_emojis = pd.Series(list(map(lambda x:target_data.columns[x], target)))
target_data.head(), train_data.head()

In [ ]:
from sklearn.model_selection import train_test_split
#train_data.drop(['text'], axis=1, inplace=True)
X_train, X_test, y_train, y_test = train_test_split(train_data, target, test_size=0.05)
X_train.reset_index(inplace=True)
X_test.reset_index(inplace=True)
X_train.shape, X_test.shape, len(y_train), len(y_test), target[:5]

In [ ]:
bad_ids = []
for emoji_index in keys:
    if sum(y_train == emoji_index) == 0:
        bad_ids.append(emoji_index)
for emoji_id in bad_ids:
    del class_weights[emoji_id]

In [ ]:
from sklearn.metrics import accuracy_score

In [ ]:
model.fit(X_train, y_train)

In [ ]:
preds = model.predict(X_test)
print(accuracy_score(y_test, preds))

In [ ]:
f_model = open('LGBM_model', 'wb')
pickle.dump(model, f_model)
f_model.close()

In [ ]:
from sklearn.metrics import recall_score
y_train_default = (y_train == 0)
model_default = lightgbm.LGBMClassifier(class_weight='balanced')
model_default.fit(X_train, y_train_default)
preds = model.predict(X_test)
preds_default = model_default.predict(X_test)
preds_result = preds == 0
true_result = y_test == 0
print(accuracy_score(preds, y_test), recall_score(preds_result, true_result))

In [ ]:
f_model_default = open('LGBM_model_default', 'wb')
pickle.dump(model_default, f_model_default)
f_model_default.close()